# dynamo query table

In [8]:
import os
import json
import time
import boto3
import tweepy
import pprint
import datetime
import numpy as np
import pandas as pd
import botocore.session
from dynamodb_json import json_util
from boto3.dynamodb.conditions import Key, Attr
# from tweet_grab

with open("../creds.json", "r") as f:
    creds = json.load(f)
    f.close()

auth = tweepy.OAuthHandler(creds['twitter-api-key'], creds['twitter-secret-key'])
auth.set_access_token(creds['twitter-access-token'], creds['twitter-secret-access'])

api = tweepy.API(auth)

client = boto3.client('dynamodb',
  # endpoint_url = "http://localhost:8000",
  aws_access_key_id=creds["aws-access-key"],
  aws_secret_access_key=creds["aws-secret-access-key"],
  region_name='us-east-2')

dynamodb = boto3.resource('dynamodb', \
            # endpoint_url = "http://localhost:8000", \
            region_name='us-east-2')

tweets = dynamodb.Table("tweets")
print(tweets.item_count)
    
print("imported modules.")

73205
imported modules.


## scan table
https://docs.aws.amazon.com/amazondynamodb/latest/developerguide/GettingStarted.Scan.html

In [10]:
done = False
start_key = None
scan_kwargs = {}
table_list = []

while not done:
    if start_key:
        scan_kwargs['ExclusiveStartKey'] = start_key
    response = tweets.scan(**scan_kwargs)
    table_list.extend(response.get('Items', []))
    start_key = response.get('LastEvaluatedKey', None)
    done = start_key is None
    print("read data for key {}".format(start_key))
    time.sleep(5)

read data for key {'user': 'EazyE23_', 'created_at': '2017-09-27T18:16:34Z'}
read data for key {'user': 'TheNovaLife_', 'created_at': '2011-07-13T17:52:07Z'}
read data for key {'user': 'mjennings007', 'created_at': '2020-12-11T19:22:03Z'}
read data for key {'user': 'TheBeltTeam', 'created_at': '2016-02-19T16:28:25Z'}
read data for key {'user': 'cd3lisi', 'created_at': '2022-05-21T13:55:13Z'}
read data for key {'user': 'Holllehhh', 'created_at': '2022-09-13T15:23:41Z'}
read data for key {'user': 'JohnnyChris', 'created_at': '2011-05-29T03:06:52Z'}
read data for key {'user': 'iblainee', 'created_at': '2016-02-12T23:01:51Z'}
read data for key {'user': 'dossermd', 'created_at': '2022-11-24T20:43:13Z'}
read data for key {'user': 'RachelClem27', 'created_at': '2017-04-22T04:46:27Z'}
read data for key {'user': 'crstelter', 'created_at': '2015-04-22T06:25:29Z'}
read data for key {'user': '__kingcrow', 'created_at': '2022-02-28T23:27:06Z'}
read data for key {'user': 'tim8oslice', 'created_at': 

## pandas

In [11]:
df = pd.DataFrame(table_list)
df.head()

,inserted_at,created_at,favorites,retweets,user,id,tweet
0,2023-05-15T15:39:57Z,2015-08-10T18:51:04Z,0,1368,missnikkay,630813633463717888,RT @BuzzFeed: puppy break https://t.co/HKKacpb1eN
1,2023-05-15T15:39:57Z,2015-08-11T19:28:55Z,2,0,missnikkay,631185546128166912,'letting go doesnt mean you dont care about so...
2,2023-05-15T15:39:57Z,2015-08-12T15:21:50Z,0,2,missnikkay,631485751805349888,RT @vetterja: I'm done. http://t.co/Ot6bub7ubE
3,2023-05-15T15:39:57Z,2015-08-12T16:41:25Z,0,0,missnikkay,631505780026400768,making a quick pit stop in VB then off to see ...
4,2023-05-15T15:39:57Z,2015-08-13T15:20:41Z,0,0,missnikkay,631847851765702656,there is pumpkin decor at target. so it begins...


In [12]:
df.shape

(73205, 7)

In [13]:
len(np.unique(df["user"]))

179

## summary stats

In [14]:
df.groupby(['user']).agg({'created_at': ['min', 'max'], 'tweet': ['count']})

created_at                       tweet
                                  min                   max count
user                                                             
420stoners_Gas   2022-05-14T11:32:14Z  2023-02-02T19:03:08Z    10
4TheLoveOfJayy   2013-03-17T22:06:25Z  2023-04-09T20:18:02Z   560
AbbasyKhan       2015-01-23T21:11:11Z  2019-08-17T15:02:26Z     5
AdamsSm45393725  2019-09-04T01:28:45Z  2019-11-03T16:16:25Z     5
AndreaPinillos4  2013-04-18T20:32:21Z  2022-08-31T03:11:03Z   469
...                               ...                   ...   ...
veronicamullen_  2016-03-26T19:08:45Z  2021-01-09T01:53:25Z   556
very_sad_sab     2019-08-13T03:40:14Z  2021-06-17T17:37:33Z    71
xPOPPYxROSEx     2017-09-08T00:02:37Z  2018-12-15T04:11:08Z   358
xeniaibanez436   2014-08-21T17:41:59Z  2014-09-29T16:17:43Z   187
ypurvazha        2022-06-07T19:24:03Z  2023-05-14T15:22:32Z   551

[179 rows x 3 columns]

In [15]:
df.to_csv('../tweets.csv', header = True, sep = ",")

## unique users

In [16]:
dynamo_users = list(np.unique(df['user']))
dynamo_users

['420stoners_Gas',
 '4TheLoveOfJayy',
 'AbbasyKhan',
 'AdamsSm45393725',
 'AndreaPinillos4',
 'AristocratAlex',
 'Ashey190',
 'AwkwardturtleAM',
 'BCroarkin5',
 'BEEEZZS',
 'BeHumanNY',
 'BeerLovesBBurg',
 'Bigdaddyge007',
 'BloodGoblet',
 'Bttmassupholed1',
 'CHsLyrics',
 'Ceresasaurus',
 'ChriMoulto',
 'DIYmindfreak',
 'DavidMa98889032',
 'DeezNutwell',
 'Dvmurray_',
 'EazyE23_',
 'Exasper8TheBand',
 'GivesNoTweets',
 'GriffM725',
 'HeadlineVC',
 'Holllehhh',
 'HunterBernsFndn',
 'IamBriAustin',
 'Jennife45385981',
 'JingliXing',
 'JohnnyChris',
 'KerryG_OT',
 'Kyotolofi',
 'LISA_CUBoulder',
 'LexiV1P',
 'Liliarose_music',
 'LindseyCherpes',
 'MBchief',
 'Mariell4Harris',
 'Marswaggo',
 'MattCabrera2',
 'MerrittBaer',
 'Miranda_Rader97',
 'MischievousMal',
 'NOSMILESGANG2',
 'NOVAproblems',
 'Namik_chocolate',
 'NicholasC_Snow4',
 'NoMintAllowed',
 'NtheSwagosphere',
 'PacSwim4',
 'Pcpetey',
 'RachelClem27',
 'ReVibeNation',
 'SarahAnneDroid',
 'SteveDoesnt',
 'SuplexPrincess',
 'Tam